In [1]:
import pandas as pd
import numpy as np
import openai
import json
import os
import requests
from dotenv import load_dotenv
from invoice_funcs import load_extract_invoice
load_dotenv()
openai.api_key=os.getenv("OPENAI_API_KEY")

### Load Invoices and Group-By Month 

In [15]:
df, concat_str = load_extract_invoice("Total Harvest Data - Alluvus.xlsx")

Date            object
Notes           object
Concatenated    object
dtype: object

In [23]:
concat_str

"01-03 Finish required cyber security training\n01-03 ODEI enterprise wide strategy presentation; Connie and Trill discussion re: PPT and next steps.\n01-03 Connie convo re: ODEI EOY report and all things ODEI - see notes.\n01-04 Annual report discussion and misc. ODEI with Connie.\n01-04 Golden Flexx update and schedule review with MR.\n01-05 Followups on metric reporting from Mirror/Mirror-Allure engagement with Conde Nast team\n01-05 Weekly touch base with team to review KPI tracking; next steps to close out 2022 and move into 2023 draft\n01-05 KPI tracking meeting\n01-05 ODEI L-Team meeting day; see notes and ongoing review thru the day with team.\n01-05 Review ODEI KPIs from team and organize with focus on year-end report; calendar review and direction to Connie N.\n01-06 ODEI L-team meeting re: KPIs and alignment; follow up meeting with CN.\n01-06 HPQC presentation and CES social inputs from CY.\n01-06 ODEI annual report correspondence review and follow up.\n01-06 AIW meeting wit

In [16]:
df['Month'] = df.copy()['Date'].str[:2]

In [22]:
df.head()

,Date,Notes,Concatenated,Month
0,01-03,Finish required cyber security training,01-03 Finish required cyber security training,01
1,01-03,ODEI enterprise wide strategy presentation; Co...,01-03 ODEI enterprise wide strategy presentati...,01
2,01-03,Connie convo re: ODEI EOY report and all thing...,01-03 Connie convo re: ODEI EOY report and all...,01
3,01-04,Annual report discussion and misc. ODEI with C...,01-04 Annual report discussion and misc. ODEI ...,01
4,01-04,Golden Flexx update and schedule review with MR.,01-04 Golden Flexx update and schedule review ...,01


In [21]:
df_by_month = df.groupby("Month")

### Classifier Function for GPT

In [ ]:
function = {
    "name": "classify_invoices",
    "description": "Given an invoice in a string of the format [MM-DD] [Note], classify it into one of the given deliverables.",
    "parameters": {
        "type": "object",
        "properties": {
            "prediction": {
                "type": "array",
                "items": {
                    "type": "String",
                    "enum": [
                        "Deliverable One: Age Discrimination Strategy – Social and Digital Content Strategy including ongoing content and editorial support across the age discrimination strategy and third-party content partnership support.",
                        "Deliverable Two: Content and Editorial Support for Age Discrimination Strategic Initiative.",
                        "Deliverable Three: Disrupt Aging/Mirror-Mirror Partnership Support.",
                        "Deliverable Four: Refinement and Redevelopment of the Workplace Pilot Program.",
                        "Deliverable Five: Refinement and Redevelopment of the Disrupt Aging Classroom Program with a focus on evolving the program to support AARP state offices.",
                        "Deliverable Six and Seven: Support Age Discrimination Strategic Initiative with a focus on combatting ageism and informing alignment.",
                        "Deliverable Eight: Tactical support on an as needed basis.",
                        "Deliverable Nine: Project management and ongoing administrative support."
                    ]
                }
                "description": "The predicted deliverables"
            }
        },
        "required": ["prediction"]
    }
}

### Function Calling Testing

In [5]:
function = {
    "name": "test_function",
    "description": "This is to test if a function works",
    "parameters": {
        "type": "object",
        "properties": {
            "test_response": {
                "type": "string",
                "description": "A response to confirm that the function was called"
            }
        },
        "required": ["test_response"]
    }
}

In [6]:
content="Call the test function"

r = openai.ChatCompletion.create(
   model="gpt-4",
   temperature=0.0,
   messages=[{"role": "user", "content": content}],
   functions=[function],
)
r

<OpenAIObject chat.completion id=chatcmpl-85LVJbwRaiEAhewYJntujPQX28w71 at 0x134ab3710> JSON: {
  "id": "chatcmpl-85LVJbwRaiEAhewYJntujPQX28w71",
  "object": "chat.completion",
  "created": 1696284993,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "test_function",
          "arguments": "{\n  \"test_response\": \"The test function has been called\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 64,
    "completion_tokens": 21,
    "total_tokens": 85
  }
}

**Media Advisory **

**All-New Dash In and Splash In ECO Car Wash Celebrate **

**Grand Opening and Baltimore’s Washington Village Community **


    **Event will honor Paul’s Place and Baltimore Green Space as part of Dash In’s Community  Engagement Program focused on Nourishing Children and Families and Reimagining  Outdoor Spaces; Dash In will donate more than $10,000 during the event **


        **What: **A day-long celebration of the grand opening of the all-new Dash In and Splash In  ECO Car Wash that also celebrates Dash In’s commitment to Baltimore’s  Washington Village community. The event will feature Dash In and Splash In  leadership honoring Paul’s Place, a 40+ year Baltimore community partner, and  Baltimore Green Space, Baltimore’s environmental land trust. Check presentations to  these organizations and Dash In’s signature ribbon cutting event – the Dash In  quesadilla pull – take place at 1:30 p.m. 

**Where: 1712 Russell Street (at the Bayard Street intersection) – Baltimore, Maryland When: Friday, September 8 – 1:30 p.m. for ribbon cutting ceremony **


    **Event Overview: **Dash In, a Wills Group company, will celebrate the grand opening of the all new Dash In and Splash In ECO Car Wash on Friday, September 8 at 1:30 p.m. with Dash In’s  signature ribbon cutting event – the Dash In quesadilla cheese pull. 


    The grand opening event celebrates Dash In’s commitment to Baltimore’s Washington Village  community with the grand opening featuring a number of activities: 


        • A $5,000 grant to Paul’s Place, a leading organization committed to leading change and  improving the quality of life in Southwest Baltimore. 


        • A $5,000 grant to Baltimore Green Spaces, Baltimore’s environmental land trust  committed to promoting vibrant neighborhoods across the city. 


        • A $500 donation to the Baltimore City Fire Foundation, focused on providing support to  the families of Baltimore City Firefighters and Paramedics. 


        • The Oriole Bird will be at the event beginning at 1:00 p.m. followed by a live performance  by the My G.I.R.L.S. All Star Cheer, Tumble and Dance Company at 3:00 p.m. 


    • Events will also be held throughout the day from 9:00 a.m. to 4:00 p.m. and include: 


        `o `Food tastings from Dash In’s chef-driven menu which connects Real Food with Real  Flavor and features the freshest ingredients. 


        `o `Free Splash In ECO car washes from 9:00 a.m. to 12 noon. 


        `o `Gift bag giveaways featuring a mystery value gift card – gift bags will be limited to  the first 50 individuals who participate with giveaways taking place at 10:00 a.m. and 2:00 p.m. 

**Dash In executives will be available for interviews at the event. **

Learn more by visiting DashIn.com

